# 1. Data reading and preprocessing
 

## 1.1 imports

In [1]:
!pip install unidecode
!pip install pytorch-pretrained-bert
!pip install pytorch-transformers
!pip install politenessr
!pip install stanza
import numpy as np

import json
#import convokit
#from convokit.politenessStrategies.politenessStrategies import PolitenessStrategies

import nltk
import pandas as pd
!python -m spacy download en
#from convokit import Corpus, download
import pickle
import sklearn
#from convokit.classifier.classifier import Classifier
#from convokit import TextParser
nltk.download('punkt')
import stanza as stanza
stanza.download('en')
from nltk.tokenize.punkt import PunktSentenceTokenizer
from spacy.lang.en import English
from tqdm import tqdm
!pip3 install -U scikit-learn

import joblib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
^C
Traceback (most recent call last):
  File "/usr/bin/pip", line 33, in <module>
    sys.exit(load_entry_point('pip==20.2', 'console_scripts', 'pip')())
  File "/usr/bin/pip", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/usr/lib/python3.8/importlib/metadata.py", line 77, in load
    module = import_module(match.group('module'))
  File "/usr/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 67

In [ ]:
# what do you want to do ?
REDO_POLITENESS=True
REDO_SENTIMENTS=True
GOOGLE_COLAB = False
if REDO_POLITENESS and REDO_SENTIMENTS:
  import json
  import requests
  # Combine all datasets int one here
  url = "https://raw.githubusercontent.com/DenisPeskov/2020_acl_diplomacy/master/data/train.jsonl"
  response = requests.get(url)
  deception= [json.loads(jline) for jline in response.content.splitlines()]

elif GOOGLE_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  with open('/content/drive/MyDrive/ada-2020-project-milestone-p3-p3_data-bayes/preprocessed_deception', 'rb') as preprocessed_deception:  
    deception = pickle.load(preprocessed_deception)
else:
  with open('./preprocessed_deception', 'rb') as preprocessed_deception:  
    deception = pickle.load(preprocessed_deception)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.2 politeness

As the original politeness classifier used for the first paper was a bit outdated and not really maintainable we decided to go for another classifier 

In [ ]:
if REDO_POLITENESS:
    from politenessr import Politenessr
    pr = Politenessr()

12/07/2020 00:13:28 - INFO - politenessr.politenessr -   Model politenessr does not exist at /usr/local/lib/python3.6/dist-packages/politenessr/politeness_model.bin. Try to download it now.
12/07/2020 00:13:28 - INFO - politenessr.download_features -   /usr/local/lib/python3.6/dist-packages/politenessr/politeness_model.bin not found in cache, downloading from http://jurgens.people.si.umich.edu/models/politeness_model.bin to /tmp/tmp78yu86ky
423140KB [02:30, 2806.35KB/s]
12/07/2020 00:15:59 - INFO - politenessr.download_features -   Model politeness_model was downloaded to a tmp file.
12/07/2020 00:15:59 - INFO - politenessr.download_features -   Copying tmp file to /usr/local/lib/python3.6/dist-packages/politenessr/politeness_model.bin.
12/07/2020 00:16:00 - INFO - politenessr.download_features -   Copied tmp model file to /usr/local/lib/python3.6/dist-packages/politenessr/politeness_model.bin.
12/07/2020 00:16:01 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/mo

In [ ]:
if REDO_POLITENESS:
  for index, game in enumerate(deception):
    politeness=[]
    for message in game['messages']:
      politeness.append(pr.predict([message]))#np.mean(nltk.tokenize.sent_tokenize(message)))
    deception[index]['politeness']=politeness

189it [00:00, 18951.07it/s]

0
4
8
ok 9
12
16
20
24
28
32
36
40
44
48
52
56
60
ok 62
64
68
72
76
80
84
ok 87
88
92
ok 96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
ok 188


In [ ]:
if REDO_POLITENESS:
  with open('preprocessed_deception', 'wb') as preprocessed_deception:
    pickle.dump(deception, preprocessed_deception)

* creation of features for politeness

## 1.3 sentiments


In [ ]:
if REDO_SENTIMENTS:
  nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')
  for index, game in tqdm(enumerate(deception)):
      deception[index]['positive_sentiments']=[]
      deception[index]['negative_sentiments']=[]
      deception[index]['neutral_sentiments']=[]
      for message in game['messages']:
        counts=[0,0,0]
        message=nlp(message)
        for sentence in message.sentences:
          counts[sentence.sentiment]+=1
        deception[index]['negative_sentiments'].append(counts[0])
        deception[index]['neutral_sentiments'].append(counts[1])
        deception[index]['positive_sentiments'].append(counts[2])

2020-12-07 14:57:07 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| sentiment | sstplus |

2020-12-07 14:57:07 INFO: Use device: cpu
2020-12-07 14:57:07 INFO: Loading: tokenize
2020-12-07 14:57:07 INFO: Loading: sentiment
2020-12-07 14:57:08 INFO: Done loading processors!

0it [00:00, ?it/s]
1it [00:23, 23.08s/it]
2it [00:32, 18.93s/it]
3it [00:38, 15.25s/it]
4it [01:07, 19.32s/it]
5it [01:09, 14.07s/it]
6it [01:20, 13.12s/it]
7it [01:21,  9.57s/it]
8it [01:24,  7.61s/it]
9it [02:07, 18.26s/it]
11it [02:22, 14.97s/it]
12it [02:26, 11.51s/it]
13it [02:28,  8.85s/it]
14it [02:28,  6.30s/it]
15it [02:30,  4.92s/it]
16it [02:35,  4.74s/it]
17it [02:35,  3.47s/it]
18it [02:37,  2.92s/it]
19it [02:37,  2.13s/it]
20it [02:48,  4.90s/it]
21it [02:48,  3.48s/it]
22it [03:01,  6.33s/it]
23it [03:23, 10.84s/it]
25it [03:27,  8.20s/it]
26it [03:33,  7.69s/it]
27it [03:34,  5.53s/it]
28it [03:48,  8.16s/it]
29it [03:50

In [ ]:
if REDO_SENTIMENTS:
  with open('preprocessed_deception', 'wb') as preprocessed_deception:
    pickle.dump(deception, preprocessed_deception)